In [2047]:
import os
import sys

import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

%load_ext autoreload
%autoreload 2

from utils.utils import (
    generate_data,
    print_project_summary,
    process_applications,
    student_assignment,
    generate_roster
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2048]:
file_location = "../data/Winter 25 Matching.xlsx"
# prioritized_students = []
prioritized_students = ["kbarbarossa@uchicago.edu"]
deprioritized_students = ["hanyunl@uchicago.edu"]
application_df, forced_assignments = process_applications(file_location, deprioritized_students, prioritized_students)

In [2049]:
technical_project_list = [
    "Morningstar", "Fermi - GNN", "Fermi - Simulations", "Center for Living Systems",
    "Building Decarbonization Coalition", "Argonne", "CMAP", "Invenergy"
]

data_to_process_dict = generate_data(application_df, technical_project_list)

Dropping 0 students because they do not have a computer and not required
Total Students available for matching: 112


In [2050]:
priority_weights = [10, 15, 25, 30]
max_students = {}
all_project_list = data_to_process_dict.pop("all_project_list")
assignment_df = student_assignment(priority_weights = priority_weights, **data_to_process_dict,
                                   max_students_dict=max_students, preassigned_students=forced_assignments,
                                   number_of_projects_to_run=None, drop_projects=["University of Northern Iowa"], verbose=False)
print(f"\nAverage Ranking: {assignment_df.Ranking.mean():.2f}")
print(assignment_df.Ranking.value_counts())
print_project_summary(assignment_df, all_project_list)

assignment_df.to_csv("../data/assignments.csv", index=False)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/hannifan/projects/the-clinic/.venv/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/sr/lrm99gp93rx47xspcx2z49gr0000gp/T/002c130cac0344a0a34cb97fb331a8b3-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/sr/lrm99gp93rx47xspcx2z49gr0000gp/T/002c130cac0344a0a34cb97fb331a8b3-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1219 COLUMNS
At line 12388 RHS
At line 13603 BOUNDS
At line 14972 ENDATA
Problem MODEL has 1214 rows, 1368 columns and 7082 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 805 - 0.00 seconds
Cgl0002I 1079 variables fixed
Cgl0003I 10 fixed, 0 tightened bounds, 8 strengthened rows, 0 substitutions
Cgl0004I processed model has 85 rows, 257 columns (257 integer (257 of which binary)) and 514 elements
Cutoff increment incre

,Project Name,Number,High Priority,High-Med Priority,Med Priority,Low Priority,Experienced,Rankings,Average Ranking
0,Argonne,4,4,0,0,0,1,"0,0,0,0",0.00
1,Morningstar,4,4,0,0,0,0,"0,0,0,1",0.25
2,International Rescue Committee,4,4,0,0,0,1,"0,0,0,1",0.25
3,Food System 6,4,4,0,0,0,1,"0,0,0,1",0.25
4,Fermi - GNN,4,4,0,0,0,1,"0,0,0,2",0.50
5,Fermi - Simulations,4,4,0,0,0,2,"0,0,1,2",0.75
6,RAFI - Grocery,4,4,0,0,0,2,"0,0,0,3",0.75
7,Internet Equity Initiative,4,4,0,0,0,0,"0,0,1,2",0.75
8,IDI - Grievances,4,4,0,0,0,1,"0,0,1,2",0.75
9,UChicago Library,4,3,0,1,0,2,"0,0,0,3",0.75


In [ ]:
# Generate roster for matching spreadsheet
roster = generate_roster(application_df, assignment_df)

roster.to_csv("../data/roster.csv", index=False)

,Project,Name,GitHub,Email,Chicago ID,Degree Program,Concentration
0,Argonne,Ana Guedes,NaN,anacguedes@uchicago.edu,NaN,Undergrad: 4th year,Data Science
1,Argonne,Angela Liu,aliu65803,aliu5@uchicago.edu,10010596Q,Undergrad: 4th year,Data Science
2,Argonne,Isaac Harlem,isaacharlem,isaacharlem@uchicago.edu,10803955L,Undergrad: 4th year,Data Science
3,Argonne,Siya Kalra,siyakalra830,siyakalra@uchicago.edu,00815932W,Undergrad: 4th year,Data Science
4,Building Decarbonization Coalition,Andres Guerrero,NaN,andresg@uchicago.edu,NaN,Undergrad: 4th year,Data Science


In [2051]:
project_to_comp = "Fermi - GNN"

list_of_not_assigned = list(assignment_df.loc[(assignment_df.Ranking.isna()), "Email Address"])
prefs_df = data_to_process_dict["ranking"].copy()
unmatched_prefs = prefs_df.loc[(prefs_df.loc[:, "Project Name"] == project_to_comp) & prefs_df.loc[:, "Email Address"].isin(list_of_not_assigned)
             & (prefs_df.Ranking < 100), :]

unmatched_info = pd.merge(unmatched_prefs, assignment_df.loc[:, ["Email Address", "Experienced", "Priority"]], how="left", on="Email Address")

matched = assignment_df.loc[(assignment_df.loc[:, "Project Assigned"] == project_to_comp), :]
display(matched.sort_values("Ranking"))

display(unmatched_info)

,Email Address,Priority,Experienced,Project Assigned,Ranking
4,alexbacro@uchicago.edu,1,False,Fermi - GNN,0
77,msaidenberg@uchicago.edu,1,False,Fermi - GNN,0
107,ygu0929@uchicago.edu,1,False,Fermi - GNN,0
100,vlois@uchicago.edu,1,True,Fermi - GNN,2.0


,Email Address,Project Name,Ranking,Experienced,Priority
0,malls@uchicago.edu,Fermi - GNN,4.0,False,4
1,victorqian@uchicago.edu,Fermi - GNN,3.0,True,3
2,wuh@uchicago.edu,Fermi - GNN,5.0,True,4
3,xuerantao@uchicago.edu,Fermi - GNN,5.0,False,4
4,yaniw@uchicago.edu,Fermi - GNN,3.0,False,4


In [2052]:
# Priority students that were not assigned to a project
assignment_df[(assignment_df["Priority"] == 1) & (assignment_df["Project Assigned"].isna())]

,Email Address,Priority,Experienced,Project Assigned,Ranking


In [2053]:
# Non-priority students that were assigned to a project
assignment_df[(assignment_df["Project Assigned"].notna()) & (assignment_df["Priority"] > 1)]

,Email Address,Priority,Experienced,Project Assigned,Ranking
17,ceden26@uchicago.edu,3,False,University of Rwanda 2,1.0
92,shaileshbolduc@uchicago.edu,3,True,UChicago Library,0
101,wanniyang@uchicago.edu,2,False,University of Rwanda,2.0
103,xiaoyue1@uchicago.edu,2,True,CMAP,0


In [2054]:
n_return = assignment_df[(assignment_df["Project Assigned"].notna()) & (assignment_df["Ranking"] == 0)].shape[0]
n_priority = assignment_df[(assignment_df["Priority"] == 1)].shape[0]
n_placed = assignment_df[(assignment_df["Project Assigned"].notna())].shape[0]

print(f"{n_return} returning students")
print(f"{n_priority} priority students")
print(f"{n_placed} total placed students")


37 returning students
68 priority students
72 total placed students


In [2055]:
# All students that were assigned
assignment_df[(assignment_df["Project Assigned"].notna())][['Email Address', 'Project Assigned']]

,Email Address,Project Assigned
1,acherman@uchicago.edu,CMAP
2,akaboski@uchicago.edu,Building Decarbonization Coalition
3,alexan@uchicago.edu,Internet Equity Initiative
4,alexbacro@uchicago.edu,Fermi - GNN
5,alexmosher@uchicago.edu,Invenergy
...,...,...
101,wanniyang@uchicago.edu,University of Rwanda
103,xiaoyue1@uchicago.edu,CMAP
107,ygu0929@uchicago.edu,Fermi - GNN
110,yurouli@uchicago.edu,Food System 6


In [2056]:
# Check a specific student by email
assignment_df[assignment_df["Email Address"] == "kbarbarossa@uchicago.edu"]

,Email Address,Priority,Experienced,Project Assigned,Ranking
55,kbarbarossa@uchicago.edu,1,False,Kids First Chicago,4.0
